Import stdPackages manually:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, pandas as pd, numpy as np, pickle, math, pyDbs, scipy
from scipy import optimize, sparse, stats
from importlib.metadata import version
from IPython.display import IFrame
_repo = 'EnergyEconGroupWork'
d = {'main': os.path.join(os.getcwd().split(_repo,1)[0],_repo),
     'curr': os.getcwd()}
d.update({'py': os.path.join(d['main'], 'py'),
          'data': os.path.join(d['main'], 'Data')})
os.chdir(d['py'])
import base, lpCompiler, lpModels, mBasicPH_storage
os.chdir(d['curr'])

For working with the model, we first import some packages and the modelclass:

In [1]:
%run ../stdPackages.ipynb
from pyDbs import *
os.chdir(os.path.join(d['curr'],'EnergyEconGroupWork', 'py'))
from mBasicPH_storage import *
os.chdir(d['curr'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Next we load the database:

In [2]:
kwargs = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables'],
        'maps': ['LoadMaps','GeneratorsMaps','StorageMaps'],
        'variable2D': ['HourlyVariation']}
path_to_data = os.path.join(d['curr'],'EnergyEconGroupWork', 'Data', 'mBasicPH_storage_Data.xlsx')
db = read.dbFromWB(path_to_data, kwargs)
readSets(db)

We then initialize and solve the model:

In [ ]:
m = mSimple(db.copy())
m()